# Spark Architecture

### Spark Cluster, Spark Execution

Spark cluster parable of classroom of teacher & students
- each table of students is like an executor & each student is a "core"
    - Each "core" is given an individual task.

Scenario 2: count total pieces in "candy bags"
- stage1: local count, each partition gets distributed to a core on a different executor.
    - driver takes result of which ever executor finishes first, then the rest of the executors commit their results after
    - Then stage1 is complete
- stage2: global count, another executor fetches all the counts from each executor after all the counts complete. These results get passed to the driver, stage2 is then complete.

//


### Shuffling & Caching

groupBy triggers a _Wide operation_. Other wide transformations include:
- `distinct, sort, join`

_Narrow transformations_ include:
- `select, filter, cast, union`

Narrow transformations = when the data is required to compute the recs in a single partition that all reside in at most 1 partition of the parent rdd.

Wide transformations = when the data is required to compute the records in a single partition that may reside in many partitions of the parent rdd.

A shuffle introduces(i.e. demarcates) stage boundaries, shuffles happen when a wide transformation happens.
Each shuffle requires a shuffle read (from disk), and a shuffle write (to disk).
1st, shuffle writes write to disk so that all subsequent shuffle reads can read those files. Shuffle writes only happen once.


//

--------------------------------------------------------------------------------

# Query Optimization

### Query Optimization

We can turn on the Adaptive Query Execution to improve the logical plan and physical plan.

//

### Demo notes

Shown an example of how a cache could accidentally block a predicate pushdown.

//

--------------------------------------------------------------------------------

# Partitioning

### Transcript
0sec:
Let's discuss partitioning. In this lesson you will understand the relationship between partitions and slots & cores.
You'll configure default shuffle partitions, describe repartition & coalesce,
match the number of partitions to the number of slots & cores,
and you'll describe dynamic coalescing of shuffle partitions in AQE.

28sec:
The spark api uses the term "core", meaning a thread available for parallel execution.

Here we refer to it as a slot to avoid confusion with the number of cores in the underlying CPU.
To which there isn't necessarily an equal number.

In most cases, if you created a cluster, you should know how many cores you have.
However, to check programmatically, you can use:
**`spark.sparkContext.defaultParallelism`**

1m2s, Cores in Cluster:
For operations like parallelize with no parent RDDs, it depends on the cluster manager.
In local mode, you'll have a number of cores on the local machine.
Mesos fine grain mode 8 and others have a total number of cores on all executor nodes or 2,
whichever is larger.

1m21s, Partitions of Data:





//

### Partitioning

Understand the relationship between partitions and slots & cores
configure default shuffle partitions
describe repartition & coalesce
match number of partitions to slots & cores
describe dynamic coalescing of shuffle partitions in AQE

In Spark, a "core" is a thread available for parallel execution.
We refer to them interchangably as slots to avoid confusion w/cores from the CPU.
`spark.sparkContext.defaultParallelism`

To get the number of partitions
`df.rdd.getNumPartitions()`

Let's say we have a pq file that was saved in 5 partitions with 8 available slots
We have 2 options to *repartition*: .coalesce(N) or .repartition(N)
pro's & con's: shuffle vs non-shuffle, even re-distribution, decrease-only vs increase-decrease

We generally want the number of partitions to a multiple of the number of available slots
e.g. if we have 4 slots ideally we should have 8 or 12 or 16... partitions

A **very general guideline** is to have each partition be roughly about 200MB... ballpark

On an executor with a reduced amount of RAM, we might need to lower the 200MB estimate. 
e.g. at 8 partitions corresponding to 4 slots, we would use close to 1.5GB 
If there are a lot of transformations that balloon each partition size, we will have problems.
So when there are a lot of transformations that balloon each partition size making the initial 
partition size, say 50MB, would probably be better. 

6m30s, Matching the number of partitions to slots
If there are 8 slots and 10 partitions, should we increase to 16 partitions or decrease to 8 partitions?
... Answer: 


10m45s, Adaptive Queary Execution



//

### Demo notes


//

### Lab notes


//